In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.DataFrame({
    'admin0': ['cntry1']*6+['cntry2']*5,
    'admin1': ['state1']*3+['state2']*3+['state3']*4+['state4'],
    'admin2': ['city1']*2+['city2']+['city3']+['city4']*2+['city5']+['city6']*3+['city6'],
    'windspeed': ['60km/h', '90km/h', '60km/h', '60km/h', '60km/h', '90km/h', '60km/h', '60km/h', '90km/h', '120km/h', '60km/h'],
    'population': [700, 210, 100, 70, 180, 370, 890, 120, 420, 360, 740]
})

In [64]:
df

,admin0,admin1,admin2,population,windspeed
0,cntry1,state1,city1,700,60km/h
1,cntry1,state1,city1,210,90km/h
2,cntry1,state1,city2,100,60km/h
3,cntry1,state2,city3,70,60km/h
4,cntry1,state2,city4,180,60km/h
5,cntry1,state2,city4,370,90km/h
6,cntry2,state3,city5,890,60km/h
7,cntry2,state3,city6,120,60km/h
8,cntry2,state3,city6,420,90km/h
9,cntry2,state3,city6,360,120km/h


In [65]:
df2 = pd.DataFrame({
    'Date': ['20130320'] * 5,
    'State': ['stA'] * 2 + ['stB'] * 2 + ['stC'],
    'City': ['ctA', 'ctB', 'ctC', 'ctD', 'ctF'],
    'SalesToday': [20, 30, 10, 40, 30],
    'SalesMTD': [400, 500, 500, 200, 300],
    'SalesYTD': [1000, 1100, 900, 1300, 800]
})

In [66]:
df2

,City,Date,SalesMTD,SalesToday,SalesYTD,State
0,ctA,20130320,400,20,1000,stA
1,ctB,20130320,500,30,1100,stA
2,ctC,20130320,500,10,900,stB
3,ctD,20130320,200,40,1300,stB
4,ctF,20130320,300,30,800,stC


In [67]:
table = pd.pivot_table(df2, values=['SalesToday', 'SalesMTD', 'SalesYTD'], 
                       index=['State', 'City'], 
                       fill_value=0, aggfunc=np.sum, dropna=True)

In [68]:
table

SalesMTD  SalesToday  SalesYTD
State City                                
stA   ctA        400          20      1000
      ctB        500          30      1100
stB   ctC        500          10       900
      ctD        200          40      1300
stC   ctF        300          30       800

In [69]:
pd.concat([
    d.append(d.sum().rename((k, 'Subtotal')))
    for k, d in table.groupby(level='State')
]).append(table.sum().rename(('All', 'Total')))

SalesMTD  SalesToday  SalesYTD
State City                                    
stA   ctA            400          20      1000
      ctB            500          30      1100
      Subtotal       900          50      2100
stB   ctC            500          10       900
      ctD            200          40      1300
      Subtotal       700          50      2200
stC   ctF            300          30       800
      Subtotal       300          30       800
All   Total         1900         130      5100

In [70]:
df3 = pd.DataFrame({
    'Employee': ['Test2', 'Test2', 'Test1', 'Test1', 'Test3', 'Test3'],
    'Account': ['Basic', 'Net', 'Basic', 'Net', 'Basic', 'Net'],
    'Currency': ['USD', 'USD', 'USD', 'USD', 'GBP', 'GBP'],
    'Amount': [3000, 2000, 4000, 3000, 5000, 4000],
    'Location': ['Airport', 'Airport', 'Town', 'Town', 'Town', 'Town']
})

In [71]:
df3

,Account,Amount,Currency,Employee,Location
0,Basic,3000,USD,Test2,Airport
1,Net,2000,USD,Test2,Airport
2,Basic,4000,USD,Test1,Town
3,Net,3000,USD,Test1,Town
4,Basic,5000,GBP,Test3,Town
5,Net,4000,GBP,Test3,Town


In [72]:
table3 = pd.pivot_table(df3, 
                        values=['Amount'],
                        index=['Location', 'Employee'],
                        columns=['Account'],
                        fill_value=0,
                        aggfunc=np.sum,
                        dropna=True)

In [73]:
table3

Amount      
Account            Basic   Net
Location Employee             
Airport  Test2      3000  2000
Town     Test1      4000  3000
         Test3      5000  4000

In [74]:
new_table3 = pd.concat([
                d.append(d.sum().rename((k, 'Subtotal')))
                for k, d in table3.groupby(level='Location')
            ]).append(table3.sum().rename(('All', 'Total')))

In [75]:
new_table3

Amount      
Account            Basic   Net
Location Employee             
Airport  Test2      3000  2000
         Subtotal   3000  2000
Town     Test1      4000  3000
         Test3      5000  4000
         Subtotal   9000  7000
All      Total     12000  9000

In [76]:
new_table3.columns

MultiIndex(levels=[['Amount'], ['Basic', 'Net']],
           codes=[[0, 0], [0, 1]],
           names=[None, 'Account'])

In [77]:
new_table3.loc[('Airport', 'Test2'), ('Amount', 'Basic')]

3000

In [78]:
new_table4 = new_table3.copy(deep=True)
new_table4.columns = new_table3.columns.droplevel(0)
new_table4

Account            Basic   Net
Location Employee             
Airport  Test2      3000  2000
         Subtotal   3000  2000
Town     Test1      4000  3000
         Test3      5000  4000
         Subtotal   9000  7000
All      Total     12000  9000

In [79]:
acc_col = new_table4.columns.ravel()
acc_col

array(['Basic', 'Net'], dtype=object)

In [80]:
new_table5 = new_table4.copy(deep=True)
new_table5

Account            Basic   Net
Location Employee             
Airport  Test2      3000  2000
         Subtotal   3000  2000
Town     Test1      4000  3000
         Test3      5000  4000
         Subtotal   9000  7000
All      Total     12000  9000

In [83]:
with pd.ExcelWriter('../data/test_table.xls') as writer:
    new_table5.to_excel(excel_writer=writer,
                        header=new_table5.columns,
                        merge_cells=False,
                        encoding='utf-8')

In [82]:
new_table5.index

MultiIndex(levels=[['Airport', 'All', 'Town'], ['Subtotal', 'Test1', 'Test2', 'Test3', 'Total']],
           codes=[[0, 0, 2, 2, 2, 1], [2, 0, 1, 3, 0, 4]],
           names=['Location', 'Employee'])